# Analisar se houve  aumento no número de ataques de tubarões

## Perguntas

### 1º  Analisar os ataques de tubarões nos últimos 10 anos? 

### 2º Qual a localização que é mais perigosa ? 

### 3º Qual a espécie mais perigosa ? 

In [1]:
!pip install openpyxl

In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame

In [3]:
df = pd.read_csv('attacks.csv',encoding='latin-')

In [4]:
df 

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.Date

0        25-Jun-2018
1        18-Jun-2018
2        09-Jun-2018
3        08-Jun-2018
4        04-Jun-2018
            ...     
25718            NaN
25719            NaN
25720            NaN
25721            NaN
25722            NaN
Name: Date, Length: 25723, dtype: object

In [ ]:
# Para padronizar os dados da coluna Date, foi utilizado o comando fillna e no lugar de vazio foi trocado pela string outros

In [6]:
df.Date.fillna('outros',inplace=True)

In [ ]:
# Nesse caso como havia a palavra Reported junto com a data, foi retirado da coluna utilizando o map para mapear a coluna 
# df.Date e realizando a operação de substituir Reported por vazio dentro de X. 

In [7]:
df.Date = df.Date.map(lambda x: x.replace('Reported', ''))

In [ ]:
# Agora será criado uma função, definindo transform_to_date de x para armazenar a tentativa de padronizar o conjunto de datas 
# com o padrão %d-%b-%Y e caso não encontre as datas necessárias, retorna nan.
# Depois define new_date com os novos valores de Date vindo do transform_to_date que recebeu o loop acima. 
# O isnull verifica se existe valores faltantes e retorna True caoso tenha
# Com os valores das datas normalizados, é possivel transformar para quarter onde será realizado uma comparação 


In [8]:
from datetime import datetime
import numpy as np
def transform_to_date(x):
    try:
        return datetime.strptime(x, '%d-%b-%Y')
    except:
        return np.nan
df['new_date'] = df.Date.map(transform_to_date)
# aqui a gente verifica tal coisa
df.new_date.isnull().value_counts(normalize=True)
df['quarter'] = df.new_date.map(lambda x: x.quarter)

In [9]:
df.Date.value_counts()

outros         19421
1957              11
1942               9
1956               8
1950               7
               ...  
24-Jun-1990        1
27-Jul-1870        1
06-Jan-2001        1
30-Apr-1830        1
Circa 1855         1
Name: Date, Length: 5433, dtype: int64

In [ ]:
# Agora será utilizado o groupby para realizar a interação da coluna 'Year', a nova coluna 'quarter' com a coluna 
# 'Case Number'. 
# No tail foi inserido o valor 42, pois se referecia ao 1º quarter de 2008 para dar 10 anos de análise de dados

In [10]:
df.groupby(['Year','quarter'])['Case Number'].count().tail(42)

Year    quarter
2008.0  1.0        16
        2.0        34
        3.0        37
        4.0        21
2009.0  1.0        31
        2.0        22
        3.0        27
        4.0        27
2010.0  1.0        20
        2.0        16
        3.0        31
        4.0        23
2011.0  1.0        18
        2.0        29
        3.0        39
        4.0        25
2012.0  1.0        25
        2.0        30
        3.0        34
        4.0        22
2013.0  1.0        20
        2.0        29
        3.0        18
        4.0        26
2014.0  1.0        17
        2.0        27
        3.0        37
        4.0        31
2015.0  1.0        26
        2.0        37
        3.0        43
        4.0        34
2016.0  1.0        24
        2.0        37
        3.0        41
        4.0        21
2017.0  1.0        21
        2.0        38
        3.0        44
        4.0        21
2018.0  1.0        18
        2.0        32
Name: Case Number, dtype: int64

In [11]:
df.groupby(['Year'])['Case Number'].count().tail(42).describe()

count     42.000000
mean      75.166667
std       35.999944
min       25.000000
25%       43.000000
50%       63.000000
75%      103.000000
max      143.000000
Name: Case Number, dtype: float64

In [12]:
df.groupby(['Year'])['Case Number'].count().tail(30)
                                                

Year
1989.0     53
1990.0     38
1991.0     38
1992.0     56
1993.0     56
1994.0     56
1995.0     76
1996.0     61
1997.0     57
1998.0     65
1999.0     66
2000.0     97
2001.0     92
2002.0     88
2003.0     92
2004.0     92
2005.0    103
2006.0    103
2007.0    112
2008.0    122
2009.0    120
2010.0    101
2011.0    128
2012.0    117
2013.0    122
2014.0    127
2015.0    143
2016.0    130
2017.0    136
2018.0     53
Name: Case Number, dtype: int64

In [13]:
df.groupby(['Year'])['Case Number'].count().sort_values().tail(30)

Year
1966.0     58
1963.0     61
1996.0     61
1998.0     65
1999.0     66
1964.0     66
1995.0     76
1961.0     78
1962.0     86
2002.0     88
2004.0     92
2001.0     92
2003.0     92
1959.0     93
1960.0     93
2000.0     97
2010.0    101
2005.0    103
2006.0    103
2007.0    112
2012.0    117
2009.0    120
2008.0    122
2013.0    122
0.0       125
2014.0    127
2011.0    128
2016.0    130
2017.0    136
2015.0    143
Name: Case Number, dtype: int64

In [14]:
df.groupby(['Year'])['Case Number'].count().sort_values(ascending=False).head(30)

Year
2015.0    143
2017.0    136
2016.0    130
2011.0    128
2014.0    127
0.0       125
2013.0    122
2008.0    122
2009.0    120
2012.0    117
2007.0    112
2006.0    103
2005.0    103
2010.0    101
2000.0     97
1960.0     93
1959.0     93
2003.0     92
2001.0     92
2004.0     92
2002.0     88
1962.0     86
1961.0     78
1995.0     76
1964.0     66
1999.0     66
1998.0     65
1996.0     61
1963.0     61
1966.0     58
Name: Case Number, dtype: int64

In [15]:
df.groupby(['Location'])['Case Number'].count().sort_values().head()

Location
                                1
Off Kawaihae                    1
Off Key Largo, Monroe County    1
Off Key West                    1
Off Kisma                       1
Name: Case Number, dtype: int64

In [29]:
df.groupby(['Location'])['Case Number'].count().sort_values(ascending=False).head(30)

Location
New Smyrna Beach, Volusia County                   163
Daytona Beach, Volusia County                       30
Ponce Inlet, Volusia County                         20
Melbourne Beach, Brevard County                     18
Myrtle Beach, Horry County                          17
Durban                                              16
Boa Viagem, Recife                                  14
Isle of Palms, Charleston County                    14
Cocoa Beach, Brevard County                         12
Ponce Inlet, New Smyrna Beach, Volusia County       12
Ahvaz, on the Karun River                           11
Palm Beach, Palm Beach County                       11
Mossel Bay                                          11
Cocoa Beach, Brevard  County                        11
Piedade                                             11
Ormond Beach, Volusia County                        11
Xai Xai                                             10
Sydney Harbor                                       10
S

In [17]:
df.groupby(['Location'])['Case Number'].count().sort_values(ascending=False).head(30)

Location
New Smyrna Beach, Volusia County                   163
Daytona Beach, Volusia County                       30
Ponce Inlet, Volusia County                         20
Melbourne Beach, Brevard County                     18
Myrtle Beach, Horry County                          17
Durban                                              16
Boa Viagem, Recife                                  14
Isle of Palms, Charleston County                    14
Cocoa Beach, Brevard County                         12
Ponce Inlet, New Smyrna Beach, Volusia County       12
Ahvaz, on the Karun River                           11
Palm Beach, Palm Beach County                       11
Mossel Bay                                          11
Cocoa Beach, Brevard  County                        11
Piedade                                             11
Ormond Beach, Volusia County                        11
Xai Xai                                             10
Sydney Harbor                                       10
S

In [18]:
x = df.groupby(['Location'])['Case Number'].count().sort_values(ascending=False).head(30)

In [19]:
x

Location
New Smyrna Beach, Volusia County                   163
Daytona Beach, Volusia County                       30
Ponce Inlet, Volusia County                         20
Melbourne Beach, Brevard County                     18
Myrtle Beach, Horry County                          17
Durban                                              16
Boa Viagem, Recife                                  14
Isle of Palms, Charleston County                    14
Cocoa Beach, Brevard County                         12
Ponce Inlet, New Smyrna Beach, Volusia County       12
Ahvaz, on the Karun River                           11
Palm Beach, Palm Beach County                       11
Mossel Bay                                          11
Cocoa Beach, Brevard  County                        11
Piedade                                             11
Ormond Beach, Volusia County                        11
Xai Xai                                             10
Sydney Harbor                                       10
S

In [20]:
x.index

Index(['New Smyrna Beach, Volusia County', 'Daytona Beach, Volusia County',
       'Ponce Inlet, Volusia County', 'Melbourne Beach, Brevard County',
       'Myrtle Beach, Horry County', 'Durban', 'Boa Viagem, Recife',
       'Isle of Palms, Charleston County', 'Cocoa Beach, Brevard County',
       'Ponce Inlet, New Smyrna Beach, Volusia County',
       'Ahvaz, on the Karun River', 'Palm Beach, Palm Beach County',
       'Mossel Bay', 'Cocoa Beach, Brevard  County', 'Piedade',
       'Ormond Beach, Volusia County', 'Xai Xai', 'Sydney Harbor', 'Sydney',
       'Singer Island, Riviera Beach, Palm Beach County', 'Nahoon',
       'Florida Keys, Monroe County', 'Jacksonville Beach, Duval County',
       'Boa Viagem Beach, Recife', 'North Beach, Durban',
       'Near Thursday Island', 'Brisbane River', 'Havana Harbor',
       'Juno Beach, Palm Beach County', 'Nahoon, East London'],
      dtype='object', name='Location')

In [21]:
df[df.Location.isin(x.index)]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23,new_date,quarter
8,2018.05.26.b,26-May-2018,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,...,2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.b,2018.05.26.b,6295.0,NaN,NaN,2018-05-26,2.0
9,2018.05.26.a,26-May-2018,2018.0,Unprovoked,USA,Florida,"Daytona Beach, Volusia County",Standing,male,M,...,2018.05.26.a-DaytonaBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.a,2018.05.26.a,6294.0,NaN,NaN,2018-05-26,2.0
27,2018.04.15.c,15-Apr-2018,2018.0,Unprovoked,BRAZIL,Pernambuco,Piedade,Swimming,Pablo de Melo,M,...,2018.04.15.c-deMelo.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.04.15.c,2018.04.15.c,6276.0,NaN,NaN,2018-04-15,2.0
39,2018.02.24,24-Feb-2018,2018.0,Unprovoked,SOUTH AFRICA,KwaZulu-Natal,Durban,Scuba Diving,Elton Polly,M,...,2018.02.24-Polly.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.02.24,2018.02.24,6264.0,NaN,NaN,2018-02-24,1.0
69,2017.10.23.b,23-Oct-2017,2017.0,Unprovoked,USA,Florida,"Juno Beach, Palm Beach County",Surfing,Jason Hartl,M,...,2017.10.23.b-Hartl.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.10.23.b,2017.10.23.b,6234.0,NaN,NaN,2017-10-23,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6263,ND.0040,1941-1945,0.0,Unprovoked,IRAN,Khuzestan Province,"Ahvaz, on the Karun River",NaN,a local dignitary,M,...,ND-0040-Local dignitary.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0040,ND.0040,40.0,NaN,NaN,NaT,NaN
6264,ND.0039,1941-1945,0.0,Unprovoked,IRAN,Khuzestan Province,"Ahvaz, on the Karun River","Standing, washing rear wheels of his ambulance...",I.A.S. C. driver,M,...,ND-0039-AmbulanceDriver.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0039,ND.0039,39.0,NaN,NaN,NaT,NaN
6286,ND.0016,Before 1921,0.0,Unprovoked,SOUTH AFRICA,KwaZulu-Natal,Durban,4 men were bathing,male,M,...,ND-0016- Durban-PostOffice.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0016,ND.0016,17.0,NaN,NaN,NaT,NaN
6289,ND.0013,No date (3 days after preceding incident) & pr...,0.0,Unprovoked,SOUTH AFRICA,KwaZulu-Natal,Durban,Fishing,a native fisherman,M,...,ND-0013-Durban-native-fisherman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0013,ND.0013,14.0,NaN,NaN,NaT,NaN


In [22]:
from datetime import datetime
import numpy as np
def transform_to_date(x):
    try:
        return datetime.strptime(x, '%d-%b-%Y')
    except:
        return np.nan
df['new_date'] = df.Date.map(transform_to_date)
# aqui a gente verifica tal coisa
df.new_date.isnull().value_counts(normalize=True)
df['quarter'] = df.new_date.map(lambda x: x.quarter)

In [23]:
df[df['new_date'].isnull()] 

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23,new_date,quarter
14,2018.05.00,May 2018,2018.0,Provoked,AUSTRALIA,Westerm Australia,Dugong Bay,Feeding sharks,Melisa Brunning,F,...,2018.05.00-Brunning.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.00,2018.05.00,6289.0,NaN,NaN,NaT,NaN
17,2018.04.30.R,30-Apr-2018,2018.0,Unprovoked,THAILAND,Hua Hin,Sai Noi Beach,Swimming,female,M,...,2018.04.30.R-Thailand.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.04.30.R,2018.04.30.R,6286.0,NaN,NaN,NaT,NaN
31,2018.04.10.R,10-Apr-2018,2018.0,Invalid,BRAZIL,Alagoas,"Praia de Sauaçuhy, Maceió",Fishing,Josias Paz,M,...,2018.04.10.R-Paz.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.04.10.R,2018.04.10.R,6272.0,NaN,NaN,NaT,NaN
59,2017.11.25.R,25-Nov-2017,2017.0,Sea Disaster,LIBYA,NaN,Gars Garabulli,2 boats capsized,31 migrants,NaN,...,2017.11.25.R-Libya.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.11.25.R,2017.11.25.R,6244.0,NaN,NaN,NaT,NaN
62,2017.11.13.R,13-Nov-2017,2017.0,Unprovoked,NaN,NaN,NaN,Surfing,Timur Yunusov,M,...,2017.11.13.R-Timur.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.11.13.R,2017.11.13.R,6241.0,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,outros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
25719,NaN,outros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
25720,NaN,outros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
25721,NaN,outros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [24]:
df[df.Location.isin(x.index)]['Species '].value_counts().head()

4' to 5' shark                     11
4' shark                            9
3' to 4' shark                      7
3' shark                            7
Shark involvement not confirmed     7
Name: Species , dtype: int64

In [25]:
import numpy as np
def normalize_Species (x):
    try:
        for specie in Species :
            if Specie in x:
                return Specie
        return 'others'
    except:
        return 'others'

df['Species '] = df['Species '].str.lower()    
df['species_normalized'] = df['Species '].map(normalize_Species)
df[df.species_normalized=='Others']['Species '].dropna()

Series([], Name: Species , dtype: object)

In [26]:
df.groupby(['Species '])['Case Number'].count().tail(42)

Species 
white shark,4.6 m [15']                                                   1
white shark?                                                              4
white sharks                                                              4
whitetip reef shark                                                       1
whitetip reef shark, 1m                                                   1
whitetip reef shark, 4' to 6'                                             1
whtietip reef shark                                                       1
wobbegong                                                                 1
wobbegong shark                                                          21
wobbegong shark                                                           1
wobbegong shark, 1 m                                                      1
wobbegong shark, 1.2 m [4']                                               2
wobbegong shark, 1.2 m [4'] k                                             1
wob

In [27]:
df.groupby(['Species '])['Case Number'].count().sort_values(ascending=False).head(5)

Species 
white shark                                           164
shark involvement prior to death was not confirmed    105
invalid                                               102
shark involvement not confirmed                        89
tiger shark                                            73
Name: Case Number, dtype: int64